In [5]:
import datetime


class FixedBond:
    def __init__(
        self,
        cusip,
        par_value,
        annual_coupon_rate,
        annual_frequency,
        maturity_date,
        issue_date,
        currency,
        first_call_date=None,
        call_price=None,
    ):
        self.cusip = cusip
        self.par_value = par_value
        self.annual_coupon_rate = annual_coupon_rate / 100.0  # Transform percentage to decimal
        self.annual_frequency = annual_frequency
        self.maturity_date = self._parse_date(maturity_date)
        self.issue_date = self._parse_date(issue_date)
        self.currency = currency
        self.first_call_date = self._parse_date(first_call_date) if first_call_date else None
        self.call_price = (
            call_price / 100.0 * par_value if call_price else None
        )  #  Transform percentage to actual amount
        self.coupon_per_period = par_value * self.annual_coupon_rate * annual_frequency  # coupon per period

    def _parse_date(self, date_str):
        # Transform 'MM/DD/YYYY' to datetime.date object
        if isinstance(date_str, str):
            month, day, year = map(int, date_str.split("/"))
            return datetime.date(year, month, day)
        return date_str

    def _add_months(self, date, months):
        # Add a specified number of months to a date and handles the end-of-month date. Just to avoid date like February 30th.
        month = date.month - 1 + months
        year = date.year + month // 12
        month = month % 12 + 1
        last_day_of_month = datetime.date(year, month, 1) + datetime.timedelta(days=32)
        last_day_of_month = last_day_of_month.replace(day=1) - datetime.timedelta(days=1)
        day = min(date.day, last_day_of_month.day)
        return datetime.date(year, month, day)

    def _generate_cash_flows(self, settlement_date, end_date, principal_amount):
        # Generate cash flow from settlement date to end date (date, amount)
        if settlement_date >= end_date:
            return []

        months_per_period = int(12 * self.annual_frequency)
        cash_flow_dates = []
        current_date = end_date

        # Backwards from the end date to generate the interest payment date
        while current_date >= self.issue_date:
            if current_date >= settlement_date and (not cash_flow_dates or cash_flow_dates[-1] != current_date):
                cash_flow_dates.append(current_date)
            prev_date = self._add_months(current_date, -months_per_period)
            if prev_date >= current_date:  # prevent infinite loop
                break
            current_date = prev_date

        cash_flow_dates.sort()  # Date sorted from smallest to largest
        cash_flows = []

        # Building cash flow and take care final installment includes principal
        for i, date in enumerate(cash_flow_dates):
            if i == len(cash_flow_dates) - 1:
                cash_flows.append((date, self.coupon_per_period + principal_amount))
            else:
                cash_flows.append((date, self.coupon_per_period))
        return cash_flows

    def price(self, settlement_date, yield_rate, call_assumption=None):
        # Calculate the price of the bond on the settlement date
        # settlement_date: 'MM/DD/YYYY'
        # yield_rate: decimal
        # call_assumption: call assumption (None, 'call', 'maturity')

        settlement_date = self._parse_date(settlement_date)
        yield_rate = float(yield_rate)

        # Dealing with worst-case yield scenarios for callable bonds
        if call_assumption is None and self.first_call_date and settlement_date <= self.first_call_date:
            price_maturity = self.price(settlement_date, yield_rate, "maturity")
            price_call = self.price(settlement_date, yield_rate, "call")
            return min(price_maturity, price_call)  # lowest price or lowest yield

        # Determine settlement date and the principal
        if call_assumption == "call" and self.first_call_date and settlement_date <= self.first_call_date:
            end_date = self.first_call_date
            principal_amount = self.call_price
        else:
            end_date = self.maturity_date
            principal_amount = self.par_value

        # If the settlement date is after the end date, return 0
        if settlement_date >= end_date:
            return 0.0

        # Generate cash flow and calculate present value
        cash_flows = self._generate_cash_flows(settlement_date, end_date, principal_amount)
        total_pv = 0.0
        for date, amount in cash_flows:
            t_years = (date - settlement_date).days / 365.0
            n_periods = t_years / self.annual_frequency  # periods
            discount_factor = 1 / (1 + yield_rate * self.annual_frequency) ** n_periods
            total_pv += amount * discount_factor

        return total_pv


# Example test
if __name__ == "__main__":
    # Take T123 as the test bond
    bond = FixedBond(
        cusip="T123",
        par_value=1000,
        annual_coupon_rate=4.50,  # 4.50%
        annual_frequency=0.5,
        maturity_date="12/31/2070",
        issue_date="01/01/2023",
        currency="CAD",
        first_call_date="01/01/2033",
        call_price=102,  # 102%
    )

    # Calculate price at 4.5% yield on January 1, 2025
    settlement = "01/01/2025"
    yield_rate = 0.045  # 4.5%
    price = bond.price(settlement, yield_rate)
    print(f"Bond's price {bond.cusip} on {settlement} is: ${price:.2f} {bond.currency}")

Bond's price T123 on 01/01/2025 is: $999.69 CAD
